# Thumbs demo - Data Collection

In this set of notebooks, we're going to attempt to solve the problem using deep learning and a single, very versatile, sensor: the camera.  You'll see how with a neural network, camera, we can teach the network a very useful behavior!

The way we'll do this is super simple:  

First, we'll manually take snapshots with thumbs up and label these images ``thumbs up``.  We save a snapshot of what the camera sees along with this label.

Second, we'll manually take snapshots with thumbs down and label these images ``free``.  Likewise, we save a snapshot along with this label.

That's all that we'll do in this notebook; data collection.  Once we have lots of images and labels, we'll upload this data to a GPU enabled machine where we'll *train* a neural network to predict thumbs up or thumbs down

### Display live camera feed

So let's get started.  First, let's initialize and display our camera. 

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to maximum size and then downscale to the desired size later.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Awesome, next let's create a few directories where we'll store all our data.  We'll create a folder ``dataset`` that will contain two sub-folders ``thumbsup`` and ``thumbsdown``, 
where we'll place the images for each scenario.

In [ ]:
import os

thumbsup_dir = 'dataset/thumbsup'
thumbsdown_dir = 'dataset/thumbsdown'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(thumbsup_dir)
    os.makedirs(thumbsdown_dir)
    print('Directories created')
except FileExistsError:
    print('Directories not created becasue they already exist')

If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many ``thumbsup`` images as ``thumbsdown`` images.  It also helps to know how many images we've collected overall.

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display

button_layout = widgets.Layout(width='128px', height='64px')
thumbsup_button = widgets.Button(description='add thumbs up', button_style='success', layout=button_layout)
thumbsdown_button = widgets.Button(description='add thumbs down', button_style='danger', layout=button_layout)
thumbsup_count = widgets.IntText(layout=button_layout, value=len(os.listdir(thumbsup_dir)))
thumbsdown_count = widgets.IntText(layout=button_layout, value=len(os.listdir(thumbsdown_dir)))

display(widgets.HBox([thumbsup_count, thumbsup_button]))
display(widgets.HBox([thumbsdown_count, thumbsdown_button]))

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(path, height=224, width=224):
    image_path = os.path.join(path, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_thumbsup():
    global thumbsup_dir, thumbsup_count
    save_snapshot(thumbsup_dir)
    thumbsup_count.value = len(os.listdir(thumbsup_dir))
    
def save_thumbsdown():
    global thumbsdown_dir, thumbsdown_count, camera
    save_snapshot(thumbsdown_dir)
    thumbsdown_count.value = len(os.listdir(thumbsdown_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
thumbsup_button.on_click(lambda x: save_thumbsup())
thumbsdown_button.on_click(lambda x: save_thumbsdown())

Great! Now the buttons above should save images to the ``thumbsup`` and ``thumbsdown`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place your hand in front of the camera with thumbsdown and press ``add thumbsdown``
2. Place your hand in front of the camera with thumbsup and press ``add thumbsup``
3. Repeat 1, 2

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below

Here are some tips for labeling data

1. Try different orientations
2. Try different lighting
4. Try different hands

It's important to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)

In [ ]:
display(image)
display(widgets.HBox([thumbsup_count, thumbsup_button]))
display(widgets.HBox([thumbsdown_count, thumbsdown_button]))

## Next

Once you've collected enough data, we'll need to train our model with these data

In [ ]:
camera.stop()